In [1]:
! python3.10 -m pip install numpy
! python3.10 -m pip install matplotlib 

'python3.10' is not recognized as an internal or external command,
operable program or batch file.
'python3.10' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import json
import numpy as np
from queue import PriorityQueue
import codecs

In [3]:
class Street:
  def __init__(self, id, name):
    self.id = id
    self.name = name
    
class Point:
  def __init__(self, id, x, y, streetList: list):
    self.id = id
    self.x = x
    self.y = y
    self.streetList = streetList
  
  def getId(self) -> int:
    return self.id
  
  def getPosition(self) -> list:
    return [self.x, self.y]
  
  def getStreetList(self) -> list:
    return self.streetList

road_id = 0
class Road:
  def __init__(self , point1: Point, point2: Point, oneWayRoad):
    global road_id
    self.id = road_id
    road_id += 1
    self.point1 = point1
    self.point2 = point2
    self.oneWayRoad = oneWayRoad
  def getPoint1(self)->Point:
    return self.point1
  def getPoint2(self)->Point:
    return self.point2
  def getLength(self):
    if self.length != -1:
      x1, y1 = self.getPoint1().getPosition()
      x2, y2 = self.getPoint2().getPosition()
      self.length = ((x1-x2)**2+(y1-y2)**2)**(1/2)
    return abs()

class Graph:
  next_point_dict = {}
  def add_road(self, road_to_add: Road):
    p1Id = road_to_add.point1.getId()
    p2Id = road_to_add.point2.getId()
    if p1Id not in self.next_point_dict:
      self.next_point_dict[p1Id] = []
    if p2Id not in self.next_point_dict:
      self.next_point_dict[p2Id] = []
    if road_to_add.oneWayRoad!= 1:
      self.next_point_dict[p2Id] += [p1Id]
    if road_to_add.oneWayRoad!= -1:
      self.next_point_dict[p1Id] += [p2Id]
  def get_next_point_dict(self):
    return self.next_point_dict

In [4]:
streetDict = {}
pointDict = {}
roadDict = []
graph = Graph()

In [5]:
def read_data_from_file():
    f = codecs.open("data.json", 'r', encoding='utf-8')
    fileContent = f.read()
    dataLoad = json.loads(fileContent)
    
    for street_id in dataLoad["street"]:
      newStreet = Street(street_id, dataLoad["street"][street_id])
      streetDict[street_id] = newStreet
    for point_id in dataLoad["point"]:
      tmpPoint = dataLoad["point"][point_id]
      newPoint = Point(int(point_id), tmpPoint["x"], tmpPoint["y"], tmpPoint["streetList"])
      pointDict[int(point_id)] = newPoint
    
    for road in dataLoad["road"]:
      newRoad = Road(pointDict[road["point1"]], pointDict[road["point2"]], road["oneWayRoad"])
      roadDict.append(newRoad)
      graph.add_road(newRoad)
    
read_data_from_file()
print(roadDict)


[<__main__.Road object at 0x00000262133AE860>, <__main__.Road object at 0x00000262133AE8C0>, <__main__.Road object at 0x00000262133AE920>, <__main__.Road object at 0x00000262133AE980>, <__main__.Road object at 0x00000262133AF9A0>, <__main__.Road object at 0x00000262133AFA00>, <__main__.Road object at 0x00000262133AFA60>, <__main__.Road object at 0x00000262133AFAC0>, <__main__.Road object at 0x00000262133AC130>, <__main__.Road object at 0x00000262133AC190>, <__main__.Road object at 0x00000262133AC1F0>, <__main__.Road object at 0x00000262133AC550>, <__main__.Road object at 0x00000262133AC5B0>, <__main__.Road object at 0x00000262133AC790>, <__main__.Road object at 0x00000262133AFEB0>, <__main__.Road object at 0x00000262133ACA60>, <__main__.Road object at 0x00000262133AC9D0>, <__main__.Road object at 0x00000262133ACA00>, <__main__.Road object at 0x00000262133AD990>, <__main__.Road object at 0x00000262133AD930>, <__main__.Road object at 0x00000262133ACC70>, <__main__.Road object at 0x000002

In [6]:
def getNearestRoad(x, y)->Road:
  pass

def calculateLength(point1: Point, point2: Point):
  x1, y1 = point1.getPosition()
  x2, y2 = point2.getPosition()
  return ((x1-x2)**2+(y1-y2)**2)**(1/2)

In [7]:

def findPath(x_start, y_start, x_target, y_target):
  startRoad = getNearestRoad(x_start, y_start)
  targetRoad = getNearestRoad(x_target, y_target)
  visitedPoint = {}
  nextPointDict = graph.get_next_point_dict()
  pathPriorityQueue = PriorityQueue()
  pathPriorityQueue.put(
    ( calculateLength(startRoad.getPoint1(), 
                    Point(-1, x_start, y_start, [])),
      startRoad.getPoint1().getId()))
  pathPriorityQueue.put(
    ( calculateLength(startRoad.getPoint2(), 
                    Point(-1, x_start, y_start, [])),
      startRoad.getPoint2().getId()))
  while not pathPriorityQueue.empty():
    tempLength, tempPointId = pathPriorityQueue.get()
    if visitedPoint[tempPointId] == True:
      continue
    visitedPoint[tempPointId] = True
    for nextPointId in nextPointDict[tempPointId]:
      pathPriorityQueue.put(
        tempLength + calculateLength(
            pointDict[tempPointId], 
            pointDict[nextPointId]
          ),
        startRoad.getPoint2().getId()
      )

In [8]:
def getNearestRoad(x_start,y_start):
    distances = np.zeros(len(roadDict))
    i = 0
    for road in roadDict:
        x1,y1 = road.getPoint1().getPosition()
        x2,y2 = road.getPoint2().getPosition()
        distance = np.abs((y2-y1)*x_start - (x2-x1)*y_start + x2*y1 - y2*x1) / np.sqrt((y2-y1)**2 + (x2-x1)**2)
        distances[i] = distance
        i = i + 1
    nearest_line_index = np.argmin(distances)
    return roadDict[nearest_line_index]
getNearestRoad(1,1)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
